# Ciw Call Centre Model from JSON

In this notebook we will create a `ciw` network model from a specification in a JSON file.

The model represents a simple urgent care call centre defined in [Monks and Harper (2023)](https://openresearch.nihr.ac.uk/articles/3-48)

* A single random arrival process
* Two activities for call triage (by an operator resource) and nurse call back (by a nurse).
* Only 40% of patients require a nurse call back by default.

<img src="img/call_centre_diagram.png" alt="call_centre" width="600"/>




## 1. Imports

### 1.1 `json2ciw` imports

**We will use:**

*  `load_call_centre_model` function that loads the built in urgent care call centre JSON file.
* `create_ciw_network` that accepts a valid JSON file that represents a DES model and returns a `ciw` model.

In [1]:
from json2ciw.datasets import load_call_centre_model
from json2ciw.engine import create_ciw_network

### 1.2 Other imports

In [2]:
import ciw
import statistics
from IPython.display import JSON

## 2. Load JSON

In [3]:
json_call_centre = load_call_centre_model()

display as collapsible JSON.  Expand to view the details.

In [4]:
JSON(json_call_centre)

<IPython.core.display.JSON object>

## 3. Convert to a Network Model

In [9]:
network = create_ciw_network(json_call_centre)

In [10]:
type(network)

ciw.network.Network

In [11]:
# Run a quick simulation to verify it works without crashing...
sim = ciw.Simulation(network)
sim.simulate_until_max_time(50)
print("Quick simulation run worked!")

Quick simulation run worked!


## Run the model for multiple replications

In [7]:
def run_replications(network, num_reps=50, runtime=1000):
    """
    Runs multiple replications of a Ciw network simulation and calculates performance metrics.
    
    Metrics Calculated:
    0. Mean number of arrivals (Count of completed visits to Node 1)
    1. Mean queuing time for operators
    2. Operator utilisation
    3. Mean number in operator queue
    4. Mean waiting time for nurse callback
    5. Nurse utilisation
    6. Number waiting in the call back queue
    """
    
    # Storage for replication results
    results = {
        'arrivals': [],
        'wait_op': [],
        'util_op': [],
        'queue_op': [],
        'wait_nurse': [],
        'util_nurse': [],
        'queue_nurse': []
    }

    # Retrieve capacity for utilization calculations
    # Note: Ciw networks use 1-based indexing logic internally, but lists are 0-based.
    # Service centers list matches the order of distributions provided.
    c_op = network.service_centres[0].number_of_servers
    c_nurse = network.service_centres[1].number_of_servers

    print(f"Starting {num_reps} replications (Runtime: {runtime})...")

    for i in range(num_reps):
        # Set seed for reproducibility per replication
        ciw.seed(i)
        
        # Initialize and run simulation
        Q = ciw.Simulation(network)
        Q.simulate_until_max_time(runtime)
        
        # Get all completed records
        recs = Q.get_all_records()
        
        # Filter records by Node ID (Ciw nodes are 1-indexed: 1=Operator, 2=Nurse)
        op_recs = [r for r in recs if r.node == 1]
        nurse_recs = [r for r in recs if r.node == 2]
        
        # --- Metric Calculations ---
        
        # 0. Number of arrivals (Count of completed visits to Entry Node)
        results['arrivals'].append(len(op_recs))
        
        # 1. Mean queuing time for operators
        # Average of waiting times for all customers who visited Node 1
        waits_op = [r.waiting_time for r in op_recs]
        results['wait_op'].append(statistics.mean(waits_op) if waits_op else 0.0)
        
        # 2. Operator Utilisation
        # Util = (Total Service Time) / (Total Time * Capacity)
        total_service_op = sum(r.service_time for r in op_recs)
        results['util_op'].append(total_service_op / (runtime * c_op))
        
        # 3. Mean number in operator queue (Lq)
        # Lq = (Total Waiting Time) / Total Time
        # This gives the time-weighted average number of people in the queue.
        total_wait_op = sum(waits_op)
        results['queue_op'].append(total_wait_op / runtime)
        
        # 4. Mean waiting time for nurse callback
        waits_nurse = [r.waiting_time for r in nurse_recs]
        results['wait_nurse'].append(statistics.mean(waits_nurse) if waits_nurse else 0.0)
        
        # 5. Nurse Utilisation
        total_service_nurse = sum(r.service_time for r in nurse_recs)
        results['util_nurse'].append(total_service_nurse / (runtime * c_nurse))
        
        # 6. Number waiting in callback queue (Lq for Nurse)
        total_wait_nurse = sum(waits_nurse)
        results['queue_nurse'].append(total_wait_nurse / runtime)

    # --- Print Aggregated Results ---
    print("\n" + "="*40)
    print(f"RESULTS OVER {num_reps} REPLICATIONS")
    print("="*40)
    print(f"0. Mean Total Arrivals:          {statistics.mean(results['arrivals']):.2f}")
    print(f"1. Mean Queue Time (Operator):   {statistics.mean(results['wait_op']):.2f}")
    print(f"2. Operator Utilisation:         {statistics.mean(results['util_op']):.2%}")
    print(f"3. Mean # in Operator Queue:     {statistics.mean(results['queue_op']):.2f}")
    print(f"4. Mean Wait Time (Nurse):       {statistics.mean(results['wait_nurse']):.2f}")
    print(f"5. Nurse Utilisation:            {statistics.mean(results['util_nurse']):.2%}")
    print(f"6. Mean # in Nurse Queue:        {statistics.mean(results['queue_nurse']):.2f}")
    print("="*40)

In [12]:
try:
  print("Here is the model built:")
  print(f"Node 1 (Operator) servers: {network.service_centres[0].number_of_servers} (Expected: 13)")
  print(f"Node 2 (Nurse) servers:    {network.service_centres[1].number_of_servers} (Expected: 9)")  
  print(sim.routers['Customer'].routers[0].probs) 

  # now run 100 multiple replications to compare to my simpy model outputs.
  run_replications(network, num_reps=100, runtime=1000)

except Exception as e:
  print(f"Error creating network: {e}")

Here is the model built:
Node 1 (Operator) servers: 13 (Expected: 13)
Node 2 (Nurse) servers:    9 (Expected: 9)
[0.0, 0.4, 0.6]
Starting 100 replications (Runtime: 1000)...

RESULTS OVER 100 REPLICATIONS
0. Mean Total Arrivals:          1644.25
1. Mean Queue Time (Operator):   3.18
2. Operator Utilisation:         92.76%
3. Mean # in Operator Queue:     5.28
4. Mean Wait Time (Nurse):       44.83
5. Nurse Utilisation:            97.17%
6. Mean # in Nurse Queue:        26.19
